## Global Imports

In [1]:
import pandas
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
london = pandas.read_csv("/home/vivek/Documents/mydata/iimb/regression/regression_input_post_code_year_month_aggregate.csv")

/home/vivek/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (142) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
london.shape

(2189743, 152)

## Dummifying Month And Cluster Columns

In [2]:
london["month"] = london["year_month_new"].map(lambda x:int(str(x)[4:]))
monthDummies = pandas.get_dummies(london["month"],prefix="month"+"_")
london = pandas.concat([london,monthDummies],axis=1)
london.drop(["month"],axis=1,inplace=True)

clusterDummies = pandas.get_dummies(london["cluster"],prefix="cluster"+"_")
london = pandas.concat([london,clusterDummies],axis=1)
london.drop(["cluster"],axis=1,inplace=True)

In [3]:
londonPrice = london[london["year_month_new"]>=200901]  ## 199501 for full data
dropList = ["borough","ward","borough_code","latitude","longitude","yearBetween",\
            "YearMon","Year","Month","ward_code",'avg_co2_emission_in_kt', 'avg_population']
londonPrice = londonPrice.drop(dropList,axis=1)

train = londonPrice[londonPrice["year_month_new"]<201401]
test = londonPrice[londonPrice["year_month_new"]>=201401]
testCols = train.columns[3:]

## Xtreme Gradient Boost

In [ ]:
from sklearn import model_selection, preprocessing
import xgboost as xgb

y_train = train["price"]
x_train = train[testCols]
x_test = test[testCols]

### Optimum Found From Grid Search
xgb_params = {
    'eta': 0.02,
    'max_depth': 8,
    'subsample': 0.5,
    'colsample_bytree': 0.2,# 
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    "booster" : "gbtree",
    'silent': 1,
    'lambda':0.7,
    'alpha':0.7
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=100,verbose_eval=50, show_stdv=False)


## Creating Model From Best Iteration

In [ ]:
num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

# Make predictions
predsTrain =model.predict(dtrain)
predsTest = model.predict(dtest)


## Calculating Train And Test MAPE

In [ ]:
trainResults = train
trainResults["Predicted"] = predsTrain
postalMeanTrain = pandas.DataFrame(trainResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
postalMeanTrain["AP"] = abs(postalMeanTrain["price"]  -  postalMeanTrain["Predicted"])/postalMeanTrain["price"]
print("Train Mape: ",postalMeanTrain.AP.mean())

testResults = test
testResults["Predicted"] = predsTest
postalMeanTest = pandas.DataFrame(testResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
postalMeanTest["AP"] = abs(postalMeanTest["price"]  -  postalMeanTest["Predicted"])/postalMeanTest["price"]
print("Test Mape: ",postalMeanTest.AP.mean())

## Running XGBoost For Every Cluster

In [ ]:
for clusNum in list(london["cluster"].unique()):
    print("cluster Num: ",clusNum)
    londonPrice = london[london["cluster"]==int(clusNum)]

    londonPrice = londonPrice[londonPrice["year_month_new"]>=200901] # 199501 for full data
    dropList = ["borough","ward","borough_code","latitude","longitude","yearBetween",\
                "YearMon","Year","Month","ward_code",'avg_co2_emission_in_kt', 'avg_population']
    londonPrice = londonPrice.drop(dropList,axis=1)

    train = londonPrice[londonPrice["year_month_new"]<201401]
    test = londonPrice[londonPrice["year_month_new"]>=201401]
    testCols = train.columns[3:]

    y_train = train["price"]
    x_train = train[testCols]
    x_test = test[testCols]


    xgb_params = {
        'eta': 0.02,
        'max_depth': 8,
        'subsample': 0.5,
        'colsample_bytree': 0.2,# 
        'objective': 'reg:linear',
        'eval_metric': 'mae',
        "booster" : "gbtree",
        'silent': 1,
        'lambda':0.7,
        'alpha':0.7
    }

    dtrain = xgb.DMatrix(x_train, y_train)
    dtest = xgb.DMatrix(x_test)

    cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=2000, early_stopping_rounds=100, show_stdv=False)
    num_boost_rounds = len(cv_output)
    model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

    # Make predictions
    predsTrain =model.predict(dtrain)
    predsTest = model.predict(dtest)

    #print (grid_gbm.best_params_)
    trainResults = train
    trainResults["Predicted"] = predsTrain
    postalMeanTrain = pandas.DataFrame(trainResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
    postalMeanTrain["AP"] = abs(postalMeanTrain["price"]  -  postalMeanTrain["Predicted"])/postalMeanTrain["price"]
    print("Train Mape: ",postalMeanTrain.AP.mean())

    testResults = test
    testResults["Predicted"] = predsTest
    postalMeanTest = pandas.DataFrame(testResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
    postalMeanTest["AP"] = abs(postalMeanTest["price"]  -  postalMeanTest["Predicted"])/postalMeanTest["price"]
    print("Test Mape: ",postalMeanTest.AP.mean())